In [ ]:
import sits.SITS as sits
import folium
from datetime import datetime
import time

In [ ]:
import geopandas as gpd
import pandas as pd

in_vect = "france_barkbeetle_lite_single.geojson"
test_2154 = sits.Vec2gdf(in_vect)
test_2154.gdf.crs.to_epsg()

In [ ]:
# calculates the boundiug box for each feature.
test_2154.set_bbox('gdf')
test_2154.bbox.crs.to_epsg()

In [ ]:
#f = folium.Figure(height=400)
#m = folium.Map(location=[55, 0], zoom_start=3).add_to(f)
#test_2154.gdf.explore(m=m, height=400, color='green')
#test_2154.bbox.explore(m=m, height=400)

In [ ]:
type(test_2154.bbox)

In [ ]:
test_process = test_2154.bbox.to_crs(4326)

test_process = pd.concat([test_process, test_process['geometry'].bounds], axis=1)
test_process['bbox_4326'] = test_process[['minx', 'miny', 'maxx', 'maxy']].values.tolist()

In [ ]:
test_3035 = test_2154.bbox.to_crs(3035)
test_3035_bounds = pd.concat([test_3035, test_3035['geometry'].bounds], axis=1)
test_3035_bounds['bbox_3035'] = test_3035_bounds[['minx', 'miny', 'maxx', 'maxy']].values.tolist()

test_process = pd.concat([test_process, test_3035_bounds['bbox_3035']], axis=1)
test_process['bbox_tuple'] = test_process.apply(lambda row: (row['bbox_4326'], row['bbox_3035']), axis=1)

In [ ]:
#%%time

#imgcoll = sits.StacAttack()
#imgcoll.searchItems(total_bbox_4326, date_start=datetime(2023, 1, 1), date_end=datetime(2023, 12, 31))

In [ ]:
#len(imgcoll.items)

In [ ]:
from dask import delayed, compute

In [ ]:
def f_test(aoi_bounds_latlong, aoi_bounds_proj, gid, rep):
    # loads time-series images in EPSG:3035
    imgcoll = sits.StacAttack()
    imgcoll.searchItems(aoi_bounds_latlong, date_start=datetime(2023, 1, 1), date_end=datetime(2023, 12, 31))
    #imgcoll.loadImgs(aoi_bounds_proj, crs_out=3035)
    imgcoll.loadPatches(aoi_bounds_proj, dimx=10, dimy=10, crs_out=3035)
    # exports time-series into csv file and netCDF file
    imgcoll.to_nc(f"/eos/jeodpp/home/users/osekenj/data/temp_data/{rep}", gid, 'patch')

In [ ]:
#for i in test_process['bbox_tuple'][:20]:
#    print (i[0],i[1])

In [ ]:
%%time
#test 01
for gid, i in enumerate(test_process['bbox_tuple'][:20]):
    f_test(i[0], i[1], gid, "test01")

In [ ]:
#%%time
#test 02
"""
fetch_dask = []
    
for gid, i in enumerate(test_process['bbox_tuple'][:20]):
    single = delayed(f_test)(i[0], i[1], gid, "test02")
    fetch_dask.append(single)

results_dask = compute(*fetch_dask)#, scheduler='processes')
"""

In [ ]:
#%%time
#test 03
"""
fetch_dask = []
    
for gid, i in enumerate(test_process['bbox_tuple'][:20]):
    single = delayed(f_test)(i[0], i[1], gid, "test03")
    fetch_dask.append(single)

results_dask = compute(*fetch_dask, scheduler='processes')
"""
# CPU times: user 74.2 ms, sys: 14.8 ms, total: 89 ms
# Wall time: 4min 8s

In [ ]:
#outdir = "/eos/jeodpp/home/users/osekenj/data/temp_data/test04b"
#test = sits.Multiproc2('image', 'nc', outdir)

In [ ]:
#for gid, i in enumerate(test_process['bbox_tuple'][:5]):
#    print(i[0], i[1], gid)
#    test.fetch_func(i[0], i[1], gid, resolution=20)

In [ ]:
#test.fetch_dask

In [ ]:
#%%time
#test.dask_compute()

In [ ]:
%%time
outdir = "/eos/jeodpp/home/users/osekenj/data/temp_data/test06_processes"
test = sits.Multiproc('patch', 'nc', outdir)
for gid, i in enumerate(test_process['bbox_tuple'][:20]):
    #print(i[0], i[1], gid)
    test.fetch_func(i[0], i[1], gid, dimx=10, dimy=10)
test.dask_compute()

In [ ]:
%%time
outdir = "/eos/jeodpp/home/users/osekenj/data/temp_data/test06_threads"
test = sits.Multiproc('patch', 'nc', outdir)
for gid, i in enumerate(test_process['bbox_tuple'][:20]):
    #print(i[0], i[1], gid)
    test.fetch_func(i[0], i[1], gid, dimx=10, dimy=10)
test.dask_compute(scheduler_type='threads')

In [ ]:
%%time
outdir = "/eos/jeodpp/home/users/osekenj/data/temp_data/test06_distributed"
test = sits.Multiproc('patch', 'nc', outdir)
for gid, i in enumerate(test_process['bbox_tuple'][:20]):
    #print(i[0], i[1], gid)
    test.fetch_func(i[0], i[1], gid, dimx=10, dimy=10)
test.dask_compute(scheduler_type='distributed')